<center>
    <h1 id='data-science-tricks' style='color:#7159c1'>🎩 Data Science Tricks 0 🎩</h1>
    <i>Getting better visualization insights and transformations of you dataset</i>
</center>

```txt
- Pivot Table
- Cross Table
- Apply Function and Map Function
- Melt Function
```

In [67]:
# ---- Imports ----
import numpy as np
import pandas as pd

# ---- Constants ----
DATASETS_PATH = ('./datasets')
SEED = (20240706)

# ---- Settings ----
np.random.seed(SEED)

<p id='0-pivot-table' style='color:#7159c1; border-bottom:3px solid #7159c1; letter-spacing:2px; font-family:JetBrains Mono; font-weight: bold; text-align:left; font-size:240%;padding:0'>0 | Pivot Table</p>

`Pivot Table` is a perfect way to see the combination of various categorical and quantitative variables. It is like a group by with a binary tree of all combinations.

Documentation: [pandas.pivot_table](https://pandas.pydata.org/docs/reference/api/pandas.pivot_table.html).

In [68]:
# ---- Reading Dataset ----
loan_df = pd.read_csv(f'{DATASETS_PATH}/loan.csv')
loan_df.columns = [column.lower() for column in loan_df.columns]

print(f'- Observations: {loan_df.shape[0]:,}')
print(f'- Variables: {loan_df.shape[1]:,}')
print('---')

loan_df.head()

- Observations: 614
- Variables: 13
---


,loan_id,gender,married,dependents,education,self_employed,applicantincome,coapplicantincome,loanamount,loan_amount_term,credit_history,property_area,loan_status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [69]:
# ---- Pivot Table ----
variables_to_pivot = ['gender', 'married', 'self_employed', 'property_area', 'coapplicantincome', 'loanamount']
loan_pivot_df = loan_df[variables_to_pivot].copy()

loan_pivot_df = loan_pivot_df.pivot_table(
    values=['coapplicantincome', 'loanamount']
    , index=['gender', 'married', 'property_area', 'self_employed']
    , aggfunc=np.mean
    , sort=True      # sorts by index values
    , margins=False  # adds 'all' row. It's great if 'aggfunc' is frequency
)

loan_pivot_df

coapplicantincome  loanamount
gender married property_area self_employed                               
Female No      Rural         No                    737.705882  119.823529
                             Yes                     0.000000  194.000000
               Semiurban     No                    543.958333  116.458333
                             Yes                   245.285714  120.666667
               Urban         No                    412.681818   96.428571
                             Yes                     0.000000  113.333333
       Yes     Rural         No                      0.000000  106.666667
                             Yes                     0.000000  121.500000
               Semiurban     No                   1733.764706  145.000000
                             Yes                     0.000000  600.000000
               Urban         No                   1554.400000  120.400000
                             Yes                     0.000000  286.000000
Male   No      Rural         No                   1573.324324  149.472222
                             Yes                   541.750000  137.750000
               Semiurban     No                   1574.027027  122.638889
                             Yes                   308.400000  141.800000
               Urban         No                   2027.000000  109.133333
                             Yes                   965.571429  216.857143
       Yes     Rural         No                   1948.495422  154.354430
                             Yes                  2098.277778  204.647059
               Semiurban     No                   1914.747664  158.132075
                             Yes                  1433.117647  135.312500
               Urban         No                   1558.257692  142.536082
                             Yes                  3131.750000  164.750000

<p id='1-cross-table' style='color:#7159c1; border-bottom:3px solid #7159c1; letter-spacing:2px; font-family:JetBrains Mono; font-weight: bold; text-align:left; font-size:240%;padding:0'>1 | Cross Table</p>

`Cross Table` is like Pivot Table, with the difference that it is used to count pairs values of category variables.

Documentation: [pandas.crosstab](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.crosstab.html).

In [70]:
# ---- Cross Table: Frequencies ----
loan_cross_frequencies_df = pd.crosstab(
    index=loan_df['gender']
    , columns=loan_df['married']
    , margins=True     # adds 'all' row and column. It is great for cross tables since frequencies are shown
    , normalize=False  # when 'True', it shows the percentages instead of the frequencies.
    # The percentage is calculated by dividing the frequency by the sum of values ('all' values)
)

loan_cross_frequencies_df

married,No,Yes,All
gender,,,
Female,80,31,111
Male,130,357,487
All,210,388,598


In [71]:
# ---- Cross Table: Percentages ----
loan_cross_percentages_df = pd.crosstab(
    index=loan_df['gender']
    , columns=loan_df['married']
    , margins=True    # adds 'all' row and column. It is great for cross tables since frequencies are shown
    , normalize=True  # when 'True', it shows the percentages instead of the frequencies.
    # The percentage is calculated by dividing the frequency by the sum of values ('all' values)
)

loan_cross_percentages_df

married,No,Yes,All
gender,,,
Female,0.133779,0.051839,0.185619
Male,0.217391,0.596990,0.814381
All,0.351171,0.648829,1.000000


<p id='2-apply-function-and-map-function' style='color:#7159c1; border-bottom:3px solid #7159c1; letter-spacing:2px; font-family:JetBrains Mono; font-weight: bold; text-align:left; font-size:240%;padding:0'>2 | Apply Function and Map Function</p>

`Apply Function` is like Map Function in JavaScript, that is, it is a great way to transform all values of a dataset. Besides, it is important give attention to `axis` parameter: when its value is 0, the data is transformed by columns; and when its value is 1, the data is transformed by rows.

`Map Function` is the former `ApplyMap Function` and the difference from Apply is that while apply transforms the whole column simultaneously, Map transforms each value of the column separately.

Documentation: [pandas.DataFrame.apply](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html) and [pandas.DataFrame.map](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.map.html).

In [80]:
# ---- Functions ----
missing_values_frequencies = lambda column: column.isnull().sum()
missing_values_percentages = lambda column: column.isnull().sum() / len(column)
replace_missing_values = lambda value: 0 if pd.isnull(value) else value

In [78]:
# ---- Counting Missing Values by Column: Frequencies ----
loan_df.apply(missing_values_frequencies, axis=0).sort_values(ascending=False)

credit_history       50
self_employed        32
loanamount           22
dependents           15
loan_amount_term     14
gender               13
married               3
loan_id               0
education             0
applicantincome       0
coapplicantincome     0
property_area         0
loan_status           0
dtype: int64

In [79]:
# ---- Counting Missing Values by Column: Percentages ----
loan_df.apply(missing_values_percentages, axis=0).sort_values(ascending=False)

credit_history       0.081433
self_employed        0.052117
loanamount           0.035831
dependents           0.024430
loan_amount_term     0.022801
gender               0.021173
married              0.004886
loan_id              0.000000
education            0.000000
applicantincome      0.000000
coapplicantincome    0.000000
property_area        0.000000
loan_status          0.000000
dtype: float64

In [83]:
# ---- Applying Map to Fill Missing Values by Zeros ----
loan_without_null_values_df = loan_df.copy()
loan_without_null_values_df = loan_without_null_values_df.applymap(replace_missing_values)

In [86]:
# ---- Checking Missing Values: Frequencies ----
loan_without_null_values_df.apply(
    missing_values_frequencies
    , axis=0
).sort_values(ascending=False)

loan_id              0
gender               0
married              0
dependents           0
education            0
self_employed        0
applicantincome      0
coapplicantincome    0
loanamount           0
loan_amount_term     0
credit_history       0
property_area        0
loan_status          0
dtype: int64

In [87]:
# ---- Checking Missing Values: Percentages ----
loan_without_null_values_df.apply(
    missing_values_percentages
    , axis=0
).sort_values(ascending=False)

loan_id              0.0
gender               0.0
married              0.0
dependents           0.0
education            0.0
self_employed        0.0
applicantincome      0.0
coapplicantincome    0.0
loanamount           0.0
loan_amount_term     0.0
credit_history       0.0
property_area        0.0
loan_status          0.0
dtype: float64

<p id='3-melt-function' style='color:#7159c1; border-bottom:3px solid #7159c1; letter-spacing:2px; font-family:JetBrains Mono; font-weight: bold; text-align:left; font-size:240%;padding:0'>3 | Melt Function</p>

<p id='reach-me' style='color:#7159c1; border-bottom:3px solid #7159c1; letter-spacing:2px; font-family:JetBrains Mono; font-weight: bold; text-align:left; font-size:240%;padding:0'>📫 | Reach Me</p>

> **Email** - [csfelix08@gmail.com](mailto:csfelix08@gmail.com?)

> **Linkedin** - [linkedin.com/in/csfelix/](https://www.linkedin.com/in/csfelix/)

> **GitHub:** - [CSFelix](https://github.com/CSFelix)

> **Kaggle** - [DSFelix](https://www.kaggle.com/dsfelix)

> **Portfolio** - [CSFelix.io](https://csfelix.github.io/).